<a href="https://colab.research.google.com/github/Facundo9c/Final-Project/blob/main/finalCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Install required libraries
!pip install torch torchvision pandas scikit-learn --quiet

In [7]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [9]:
# Generate a synthetic dataset
def generate_synthetic_data(num_samples=1000):
    np.random.seed(42)
    team1_odds = np.random.uniform(1.5, 3.0, num_samples)
    team2_odds = np.random.uniform(1.5, 3.0, num_samples)
    team1_performance = np.random.uniform(50, 100, num_samples)
    team2_performance = np.random.uniform(50, 100, num_samples)
    outcome = (team1_performance > team2_performance).astype(int)  # 1 if team1 wins, 0 otherwise

    data = pd.DataFrame({
        'team1_odds': team1_odds,
        'team2_odds': team2_odds,
        'team1_performance': team1_performance,
        'team2_performance': team2_performance,
        'outcome': outcome
    })
    return data

In [10]:
# Create the dataset
data = generate_synthetic_data()

In [11]:
# Check the first few rows of the dataset
print("Sample Data:")
print(data.head())


Sample Data:
   team1_odds  team2_odds  team1_performance  team2_performance  outcome
0    2.061810    1.777699          63.085284          83.635150        0
1    2.926071    2.312851          62.348940          89.834070        0
2    2.597991    2.809419          95.312729          62.523395        1
3    2.397988    2.598337          62.477310          81.243705        0
4    1.734028    2.709842          63.597486          78.587299        0


In [12]:
# Split features and target
X = data[['team1_odds', 'team2_odds', 'team1_performance', 'team2_performance']]
y = data['outcome']

In [13]:
# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [14]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

In [16]:
# Define a modified AlexNet for tabular data
class AlexNetTabular(nn.Module):
    def __init__(self, input_features):
        super(AlexNetTabular, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_features, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [17]:
# Initialize model, loss, and optimizer
model = AlexNetTabular(input_features=X_train.shape[1])
criterion = nn.BCELoss()  # Binary Cross Entropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [18]:
# Training loop
epochs = 20
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    predictions = model(X_train_tensor)
    loss = criterion(predictions, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [5/20], Loss: 0.6229
Epoch [10/20], Loss: 0.5440
Epoch [15/20], Loss: 0.4563
Epoch [20/20], Loss: 0.3796


In [19]:
# Evaluate the model
model.eval()
with torch.no_grad():
    test_predictions = model(X_test_tensor)
    test_predictions = (test_predictions > 0.5).float()
    accuracy = (test_predictions == y_test_tensor).float().mean()
    print(f"Test Accuracy: {accuracy.item() * 100:.2f}%")

Test Accuracy: 96.00%


In [20]:
# Inference: Betting recommendation
def recommend_bet(odds_team1, odds_team2, performance_team1, performance_team2):
    features = scaler.transform([[odds_team1, odds_team2, performance_team1, performance_team2]])
    features_tensor = torch.tensor(features, dtype=torch.float32)
    with torch.no_grad():
        prediction = model(features_tensor).item()
        recommendation = "Bet on Team 1" if prediction > 0.5 else "Bet on Team 2"
    return recommendation, prediction


In [21]:
# Example usage
odds_team1 = 2.1
odds_team2 = 1.8
performance_team1 = 85
performance_team2 = 78

recommendation, confidence = recommend_bet(odds_team1, odds_team2, performance_team1, performance_team2)
print(f"Recommendation: {recommendation}, Confidence: {confidence:.2f}")

Recommendation: Bet on Team 1, Confidence: 0.52


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
